Sources of board info:

https://github.com/adafruit/arduino-board-index/tree/gh-pages/boards  
https://github.com/arduino/Arduino/wiki/Unofficial-list-of-3rd-party-boards-support-urls  
https://github.com/arduino/ArduinoCore-samd  
https://github.com/arduino/ArduinoCore-avr  

I'm going to start with just arduino avr

In [46]:
!pip install packaging

  Using cached packaging-20.3-py2.py3-none-any.whl (37 kB)
  Using cached pyparsing-2.4.6-py2.py3-none-any.whl (67 kB)


In [2]:
from serial.tools import list_ports
import serial
import re
from boards import board_dict
def check_ports():
    not_arduinos = []
    arduinos = []
    for connection in list_ports.comports():
        port,hwid,desc = connection
        vid_pid = re.search(r"(?<=VID\:PID\=)[0-9|A-Z|a-z]{4}\:[0-9|A-Z|a-z]{4}", desc)
        vid_pid = None if vid_pid is None else vid_pid.group()
        if vid_pid is None:
            not_arduinos.append(connection)
        else:
            try:
                board = board_dict[vid_pid]
                arduinos.append((board,connection))
            except KeyError:
                not_arduinos.append(connection)
    return arduinos, not_arduinos
# list_o_ports = list_ports.comports()

# portname = None
# for port,hwid,desc in list_o_ports:
#     vid = re.search(r"(?<=VID\:PID\=)[0-9|A-Z|a-z]*", desc)
#     pid = re.search(r"(?<=VID\:PID\=[0-9|A-Z|a-z]{4}\:)[0-9|A-Z|a-z]*", desc)
#     vid = None if vid is None else vid.group()
#     pid = None if pid is None else pid.group()
#     #if vid == '10C4' and pid == 'EA60':
#     if vid == '2341' and pid == '0043':
#         portname = port
# if portname is None:
#     raise ConnectionError("Couldn't find your Arduino -- Check the USB Connection")

# arduino = serial.Serial(portname)
# if arduino.is_open:
#     print("successfully connected")
# else:
#     print(f"couldn't connect on port {portname}")


In [3]:
check_ports()

([('Arduino Uno', <serial.tools.list_ports_linux.SysFS at 0x7f2dec421b20>)],
 [<serial.tools.list_ports_linux.SysFS at 0x7f2dec421a30>])

## Exploring extracting

In [231]:
from packaging.version import parse as version_parse


In [6]:
board_dict = {}
with open('boards.txt','r') as f:
    for line in f:
        line = line.strip()
        if len(line)==0:
            continue
        if '.name=' in line:
            name = line[line.index("=")+1:]
            print(name)
        elif '.vid.' in line:
            vid_pid = line[-4:] +":"+ next(f)[-5:-1]
            board_dict[vid_pid] = name

Arduino Yún
Arduino Uno
Arduino Duemilanove or Diecimila
Arduino Nano
Arduino Mega or Mega 2560
Arduino Mega ADK
Arduino Leonardo
Arduino Leonardo ETH
Arduino Micro
Arduino Esplora
Arduino Mini
Arduino Ethernet
Arduino Fio
Arduino BT
LilyPad Arduino USB
LilyPad Arduino
Arduino Pro or Pro Mini
Arduino NG or older
Arduino Robot Control
Arduino Robot Motor
Arduino Gemma
Adafruit Circuit Playground
Arduino Yún Mini
Arduino Industrial 101
Linino One
Arduino Uno WiFi


In [7]:
# board_dict

In [8]:
arduinos, not_arduinos = check_ports()

In [9]:
for a in arduinos:
    print(a[0],a[1][0])

Arduino Uno /dev/ttyACM0


In [10]:
for n in not_arduinos:
    print(n[0],n[1][0])

/dev/ttyS0 t


In [11]:
not_arduinos

## working through json nonsense

In [20]:
import json
import urllib

In [16]:
json.decode("https://adafruit.github.io/arduino-board-index/package_adafruit_index.json")

AttributeError: module 'json' has no attribute 'decode'

In [90]:
url = "https://raw.githubusercontent.com/DFRobot/DFRobotDuinoBoard/master/package_dfrobot_iot_mainboard.json"
zoop = json.loads(urllib.request.urlopen(url).read())

JSONDecodeError: Expecting value: line 45 column 7 (char 1312)

In [55]:
parse("1.1.1")>parse("1.4.2")

False

In [56]:
platform1 = zoop['packages'][0]['platforms']

{'avr': {'version': '1.4.13',
  'url': 'https://adafruit.github.io/arduino-board-index/boards/adafruit-avr-1.4.13.tar.bz2'},
 'samd': {'version': '1.5.11',
  'url': 'https://adafruit.github.io/arduino-board-index/boards/adafruit-samd-1.5.11.tar.bz2'},
 'wiced': {'version': '0.6.6',
  'url': 'https://adafruit.github.io/arduino-board-index/boards/adafruit-wiced-0.6.6.tar.bz2'},
 'nrf52': {'version': '0.19.0',
  'url': 'https://adafruit.github.io/arduino-board-index/boards/adafruit-nrf52-0.19.0.tar.bz2'}}

In [99]:
headers = {'User-Agent':'Mozilla/5.0 (Windows NT 6.1; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/62.0.3202.89 Safari/537.36'}
def getResponse(url):
    operUrl = urllib.request.urlopen(url)
    if(operUrl.getcode()==200):
        data = operUrl.read()
        jsonData = json.loads(data)
    else:
        print(f"Error receiving data for {url}")
    return jsonData

In [100]:
unoffical_list = urllib.request.urlretrieve("https://raw.githubusercontent.com/wiki/arduino/Arduino/Unofficial-list-of-3rd-party-boards-support-urls.md","Unofficial-list-of-3rd-party-boards-support-urls.md")


In [124]:
find_json_re = r"(https?:\/\/[a-zA-Z0-9-_./]+.json)"
find_name_re = r"package_.+_index.json"
links = []
for line in open('Unofficial-list-of-3rd-party-boards-support-urls.md'):
    search = re.search(find_json_re,line)
    if search:
        link = search.group(0)
        name = re.search(find_name_re,link)
        if name:
            links.append((name.group(0),link))
        else:
            #json links that don't conform
            print(link)


https://raw.githubusercontent.com/DFRobot/DFRobotDuinoBoard/master/package_dfrobot_iot_mainboard.json


In [107]:
from urllib.error import HTTPError, URLError
from json import JSONDecodeError
from socket import gaierror

In [126]:
jsons = []
for name,link in links:
#     print(link)
    try:
        jsons.append(getResponse(link))
        with open("JSON/"+name, 'w') as f:
            json.dump(jsons[-1],f)
    except URLError:
        print('url:', link)
    except HTTPError:
        print("http:",link)
    except JSONDecodeError:
        print('json:', link)
    except gaierror:
        print('gai:', link)


url: http://www.arducam.com/downloads/ESP8266_UNO/package_ArduCAM_index.json
url: http://www.arducam.com/downloads/ESP32_UNO/package_ArduCAM_ESP32S_UNO_index.json
url: https://git.oschina.net/dfrobot/FireBeetle-ESP32/raw/master/package_esp32_index.json
url: http://rfduino.com/package_rfduino_index.json


In [109]:
len(jsons)

103

In [116]:
with open ('yikes.json','w') as fp:
    json.dump(jsons[0],fp)

In [117]:
s = "http://www.arducam.com/downloads/ESP8266_UNO/package_ArduCAM_index.json"

In [118]:
s

'http://www.arducam.com/downloads/ESP8266_UNO/package_ArduCAM_index.json'

## Given json file load all the info

In [228]:
platforms

[{'boards': [{'name': 'Adafruit Flora'},
   {'name': 'Adafruit Gemma 8MHz'},
   {'name': 'Adafruit Trinket 8MHz'},
   {'name': 'Adafruit Trinket 16MHz'},
   {'name': 'Pro Trinket 5V/16MHz (USB)'},
   {'name': 'Pro Trinket 3V/12MHz (USB)'},
   {'name': 'Pro Trinket 5V/16MHz (FTDI)'},
   {'name': 'Pro Trinket 3V/12MHz (FTDI)'}],
  'architecture': 'avr',
  'name': 'Adafruit AVR Boards',
  'category': 'Adafruit',
  'version': '1.1.0',
  'archiveFileName': 'adafruit-1.1.0.tar.bz2',
  'checksum': 'SHA-256:4e72d4267d9a8d86874edcd021dc661854a5136c0eed947a6fe10366bc51e373',
  'help': {'online': 'https://forums.adafruit.com'},
  'url': 'https://adafruit.github.io/arduino-board-index/boards/adafruit-1.1.0.tar.bz2',
  'toolsDependencies': [{'name': 'avr-gcc',
    'packager': 'arduino',
    'version': '4.8.1-arduino5'},
   {'name': 'avrdude', 'packager': 'arduino', 'version': '6.0.1-arduino5'}],
  'size': '29455'},
 {'boards': [{'name': 'Adafruit Flora'},
   {'name': 'Adafruit Gemma 8MHz'},
   {'na

In [230]:
platforms = jsons[0]['packages'][0]['platforms']

arch_urls = {}
for i,p in enumerate(platforms):
    arch = p['name'] +' - ' + p['architecture']
    if arch in arch_urls.keys():
        if version_parse(p['version']) > version_parse(arch_urls[arch]['version']):
            arch_urls[arch] = {'version':p['version'],'url':p['url'],'name':p['url'][p['url'].rfind('/')+1:]}
        else:
            continue
    else:
        arch_urls[arch] = {'version':p['version'],'url':p['url'],'name':p['url'][p['url'].rfind('/')+1:]}
arch_urls

{'Adafruit AVR Boards - avr': {'version': '1.4.13',
  'url': 'https://adafruit.github.io/arduino-board-index/boards/adafruit-avr-1.4.13.tar.bz2',
  'name': 'adafruit-avr-1.4.13.tar.bz2'},
 'Adafruit SAMD Boards - samd': {'version': '1.5.11',
  'url': 'https://adafruit.github.io/arduino-board-index/boards/adafruit-samd-1.5.11.tar.bz2',
  'name': 'adafruit-samd-1.5.11.tar.bz2'},
 'Adafruit WICED - wiced': {'version': '0.6.6',
  'url': 'https://adafruit.github.io/arduino-board-index/boards/adafruit-wiced-0.6.6.tar.bz2',
  'name': 'adafruit-wiced-0.6.6.tar.bz2'},
 'Adafruit nRF52 - nrf52': {'version': '0.19.0',
  'url': 'https://adafruit.github.io/arduino-board-index/boards/adafruit-nrf52-0.19.0.tar.bz2',
  'name': 'adafruit-nrf52-0.19.0.tar.bz2'}}

In [211]:
s = 'https://adafruit.github.io/arduino-board-index/boards/adafruit-nrf52-0.19.0.tar.bz2'
s[s.rfind('/')+1:]

'adafruit-nrf52-0.19.0.tar.bz2'

In [150]:
with open('JSON/package_simba_avr_index.json', 'r') as f:
    j = json.load(f)

In [151]:
arch_urls = {}
p0 = j['packages'][0]
for package in j['packages']:
    for i,p in enumerate(package['platforms']):
        arch = p['architecture']
        if arch in arch_urls.keys():
            if version_parse(p['version']) > version_parse(arch_urls[arch]['version']):
                arch_urls[arch] = {'version':p['version'],'url':p['url']}
            else:
                continue
        else:
            arch_urls[arch] = {'version':p['version'],'url':p['url']}

In [215]:
for k in arch_urls.keys()

dict_keys(['avr', 'samd', 'wiced', 'nrf52'])

  Using cached requests-2.23.0-py2.py3-none-any.whl (58 kB)
  Using cached chardet-3.0.4-py2.py3-none-any.whl (133 kB)
  Using cached idna-2.9-py2.py3-none-any.whl (58 kB)
  Using cached urllib3-1.25.8-py2.py3-none-any.whl (125 kB)


In [172]:
import tarfile

In [173]:
tf = tarfile.open('yikes.tar.bz2')

In [177]:
info = tf.getmembers()[0]

In [208]:
f = tf.extractfile([info for info in tf.getmembers() if ('boards.txt' in info.name)][0])
with open('wow.txt','wb') as wow:
    iterable = iter(f.readlines())
    for line in iterable:
        wow.write(line)

        line = line.decode().strip()
        if '.name=' in line:
            name = line[line.index("=")+1:]
            print(name)
        elif '.vid.' in line:
            new_line = next(iterable)
            wow.write(new_line)
            vid_pid = line[-4:] +":"+ new_line.decode().strip()[-5:-1]
            board_dict[vid_pid] = name
# with open('wow','w') as wow:
#     wow.write(f)

Adafruit Flora
Adafruit Feather 32u4
Adafruit Feather 328P
Adafruit Gemma (ATtiny85 @ 8MHz)
Adafruit Trinket (ATtiny85 @ 8MHz)
Adafruit Trinket (ATtiny85 @ 16MHz)
Adafruit Metro
Pro Trinket 5V/16MHz (USB)
Pro Trinket 3V/12MHz (USB)
Pro Trinket 5V/16MHz (FTDI)
Pro Trinket 3V/12MHz (FTDI)
Adafruit Circuit Playground Classic
Adafruit ItsyBitsy 32u4 5V 16MHz
Adafruit ItsyBitsy 32u4 3V 8MHz
Adafruit Bluefruit Micro
Adafruit 32u4 Breakout


In [232]:
.replace('/','-')

'adafruit-avr-1.4.13/boards.txt'

In [233]:
'adafruit-avr-1.4.13/boards.txt'.replace

'adafruit-avr-1.4.13-boards.txt'

In [201]:
f.readlines()

[]

In [238]:
[(tf.extractfile(info),info.name) for info in tf.getmembers() if ('boards.txt' in info.name)]

[(<ExFileObject name=None>, 'adafruit-avr-1.4.13/boards.txt')]